In [ ]:
pip install sklearn

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.impute import SimpleImputer
from sklearn.impute import KNNImputer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import GridSearchCV



Memory Reduction

In [ ]:
def reduce_memo(df):
    for col in df.columns:
        if df[col].dtype =="float64":
            df[col]=df[col].astype('float32')

        elif df[col].dtype==('int64'):
            df[col]=df[col].astype('int32')
    return df

LOAD THE DATA

In [ ]:
departments= pd.read_csv("D:\DATASET\departments.csv")
orders=pd.read_csv("D:\DATASET\orders.csv")
order_product=pd.read_csv("D:\DATASET\order_products__prior.csv")
product=pd.read_csv("D:\DATASET\products.csv")
aisle=pd.read_csv("D:\DATASET/aisles.csv")


In [ ]:
orders= reduce_memo(orders)
departments=reduce_memo(departments)
order_product=reduce_memo(order_product)
product=reduce_memo(product)
aisle=reduce_memo(aisle)

Look at the Data

In [ ]:
aisle

In [ ]:
order_product.head()

We will join our data into one df

In [ ]:
complete_df=pd.merge(order_product,product,how='inner',on='product_id')
complete_df=pd.merge(complete_df,departments,how='inner',on='department_id')
complete_df=pd.merge(complete_df,aisle,how='inner',on='aisle_id')
complete_df=pd.merge(complete_df,orders,how='inner',on='order_id')
complete_df.head()
sample_df=complete_df.sample(200000,random_state=42)

EDA


Look at data

In [ ]:
complete_df.info()

Summary of Data

In [ ]:
complete_df.describe()

Distrubution of days since prior order

In [ ]:
plt.figure(figsize=(10,5))
sns.histplot(complete_df['days_since_prior_order'],bins=30,kde=False)
plt.title("Distrubution of days Since Prior Order")
plt.xlabel("Days Since Prior Order")
plt.ylabel("Count of Orders")
plt.show()

Top 30 Product Orderd

In [ ]:
most_ordered=complete_df['product_name'].value_counts().head(30)
plt.figure(figsize=(12,6))
sns.barplot(x=most_ordered,y=most_ordered.index)
plt.xlabel("count of orders")
plt.title("Top 30 Product Ordred")

Day Of Week Plot

In [ ]:
sns.histplot(complete_df['order_dow'],bins=7,kde=False)

Order Per department

In [ ]:
top_departments=complete_df['department'].value_counts()
plt.figure(figsize=(12,6))
sns.barplot(x=top_departments,y=top_departments.index)
plt.xlabel("count of orders")
plt.title("ORDER PER DEPARTMENT")
plt.show()

Number of orders per users

In [ ]:
user_order=complete_df.groupby('user_id')['order_number'].max()
plt.figure(figsize=(10,5))
sns.histplot(user_order,bins=30)
plt.title("Number of Orders Per User")
plt.show()


Reorderd Vs orderd


In [ ]:
plt.figure(figsize=(6,6))
sns.countplot(x=complete_df['reordered'])
plt.title("Reorderd vs Not Reorderd")
plt.show()

The Top 20 aisle

In [ ]:
top_aisle=complete_df['aisle'].value_counts().head(20)
plt.figure(figsize=(12,6))
sns.barplot(x=top_aisle,y=top_aisle.index)
plt.title("Top 20 aisle")
plt.xlabel("count (Number of products)")
plt.ylabel("aisle")
plt.show()

REorder VS Order number

In [ ]:
order_ordernum=complete_df.groupby('order_number')['reordered'].mean().reset_index()
plt.figure(figsize=(10,5))
sns.lineplot(data=order_ordernum,x='order_number',y='reordered')
plt.title("Reorder Rate VS Order Number")
plt.show()

Handle Numrical Features

In [ ]:
num_cols=complete_df.select_dtypes(include=['int32','float32','int64','float64']).columns
num_cols

Handle Categorical Features

In [ ]:
cat_cols=complete_df.select_dtypes(include=['object']).columns
cat_cols

Correlation For Numrical Featuers ("Heatmap")

In [ ]:
plt.figure(figsize=(10,5))
correlation_matrix=complete_df[num_cols].corr()
sns.heatmap(correlation_matrix,annot=True,cmap='coolwarm')
plt.title("Numerical Feature Correlation")
plt.show()

Correlation For Numrical Featuers ("pairwise scatter")

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
plt.figure(figsize=(8,9))
sample_df=complete_df.sample(1000,random_state=42)
sns.pairplot(sample_df[num_cols])
plt.title("Pairplot of Numerical Features")
plt.show()

Time Of Day plot

In [ ]:
whole_day=complete_df['order_hour_of_day'].value_counts().sort_index()
plt.figure(figsize=(12,6))
sns.barplot(x=whole_day.index,y=whole_day.values)
plt.title("Orders by Hour of Day")
plt.show()


Orders in the Whole Week

In [ ]:
whole_week=complete_df['order_dow'].value_counts().sort_index()
plt.figure(figsize=(10,5))
sns.barplot(x=whole_week.index,y=whole_week.values)
plt.title("Orders by Day of Week")
plt.show()

In [ ]:
# --- IGNORE ---

HANDLE MISSING VALUE

In [ ]:
missing_value= complete_df.isnull().sum()
missing_value

In [ ]:
missing_count=complete_df.isnull().sum()
missing_count=missing_count[missing_count>0].sort_values(ascending=False)
plt.figure(figsize=(12,4))
sns.barplot(x=missing_count.index,y=missing_count.values)
plt.xticks(rotation=90)
plt.title("Missing Value Per Column")
plt.show()

In [ ]:
print ("..")

3. Cleaning & Imputation

Median Imputation

In [ ]:
from sklearn.impute import SimpleImputer
median_df=complete_df.copy()
median_imputer=SimpleImputer(strategy='median')
median_df['days_since_prior_order']=median_imputer.fit_transform(median_df[['days_since_prior_order']])
median_df.isnull().sum()

 Median Visualize

In [ ]:
median_df.isnull().sum()
plt.figure(figsize=(10,5))
sns.histplot(median_df['days_since_prior_order'], bins=30, kde=False)
plt.title("Histogram of Days Since Prior Order(Median Imputation)")
plt.xlabel("Days")
plt.ylabel("Frequency")
plt.show()

Most Frequent (Mode) Imputation

In [ ]:
from sklearn.impute import SimpleImputer
freq_df=complete_df.copy()
freq_imputer = SimpleImputer(strategy='most_frequent')
freq_df['days_since_prior_order'] = freq_imputer.fit_transform(freq_df[['days_since_prior_order']])
freq_df.isnull().sum()

Most Frequent (Mode) Visualize

In [ ]:
plt.figure(figsize=(10,5))
sns.histplot(freq_df['days_since_prior_order'], bins=30, kde=False)
plt.title("Histogram of Days Since Prior Order (Mode Imputation)")
plt.xlabel("Days")
plt.ylabel("Frequency")
plt.show()

Sentinel Imputation (الي رح نعتمده)

In [ ]:
from sklearn.impute import SimpleImputer
sentinel_df=complete_df.copy()
sentinel_vl=0
sentinel_df['days_since_prior_order']=sentinel_df['days_since_prior_order'].fillna(sentinel_vl)
sentinel_df.isnull().sum()



Sentinel Visualize

In [ ]:
plt.figure(figsize=(10,5))
sns.histplot(sentinel_df['days_since_prior_order'], bins=30, kde=False)
plt.title("Histogram of Days Since Prior Order (Sentinel Value Imputation)")
plt.xlabel("Days")
plt.ylabel("Frequency")
plt.show()

Model-Based Imputation (KNN Imputer)

In [ ]:
from sklearn.impute import KNNImputer
sample_knn = complete_df.sample(100000, random_state=42)
knn_imputer = KNNImputer(n_neighbors=5)
knn_sample_df = sample_knn.copy()
knn_sample_df[['days_since_prior_order']] = knn_imputer.fit_transform(knn_sample_df[['days_since_prior_order']])
knn_sample_df.isnull().sum()

Model Based Visualize

In [ ]:
plt.figure(figsize=(10,5))
sns.histplot(knn_sample_df['days_since_prior_order'], bins=30, kde=False)
plt.title("Histogram of Days Since Prior Order (KNN Imputation)")
plt.xlabel("Days")
plt.ylabel("Frequency")
plt.show()

In [ ]:
complete_df['days_since_prior_order']=complete_df['days_since_prior_order'].fillna(sentinel_vl)

3-Outlier Detection & Treatment (Z-Score Method)

In [ ]:
outlier_columns = []
for col in complete_df.select_dtypes(include=['int', 'float']):
    Q1 = complete_df[col].quantile(0.25)
    Q3 = complete_df[col].quantile(0.75)
    IQR = Q3 - Q1

    if ((complete_df[col] < (Q1 - 1.5 * IQR)) | (complete_df[col] > (Q3 + 1.5 * IQR))).any():
        outlier_columns.append(col)

print("Outlier columns:", outlier_columns)


In [ ]:
import numpy as np
outlier_columns=['add_to_cart_order','order_number']
mean_val =complete_df[outlier_columns].mean()
std_val  = complete_df[outlier_columns].std()
z_score = (complete_df[outlier_columns]- mean_val) / std_val
z_score



Extracting Outliers

In [ ]:
outlier_z = z_score[(z_score > 3) | (z_score < -3)]
print(outlier_z.shape)
len(outlier_z)

Boxplot Before Removing Outliers (Z-score) Visualize

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
plt.figure(figsize=(10,4))
sample_df=complete_df.sample(20000,random_state=42)
sns.boxplot(data=sample_df[outlier_columns])
plt.title("Boxplot Before Removing Outliers (Z-score)")
plt.show()


Histogram Before Removing Outliers (Z-score) Visualize

In [ ]:
plt.figure(figsize=(10,4))
sns.histplot(data=sample_df[outlier_columns], bins=30, kde=False,palette="coolwarm")
plt.title("Histogram Before Removing Outliers (Z-score)")
plt.xlabel("Days")
plt.ylabel("Frequency")
plt.show()

Removing Outliers

In [ ]:
clean_z_df =sample_df[(z_score <= 3) & (z_score >= -3)]
clean_z_df.shape

Boxplot After Removing Outliers

In [ ]:
plt.figure(figsize=(10,4))
sns.boxplot(data=clean_z_df[outlier_columns])
plt.title("Boxplot After Removing Outliers (Z-score)")
plt.show()


Histogram After Removing Outliers

In [ ]:
plt.figure(figsize=(10,5))
sns.histplot(data=clean_z_df[outlier_columns], bins=30, kde=False)
plt.title("Histogram After Removing Outliers (Z-score)")
plt.xlabel("Days")
plt.ylabel("Frequency")
plt.show()

4-Encoding Categorical Variables

One-Hot Encoding (for low-cardinality categories)

In [ ]:
low_card= ['department', 'aisle']
one_hot = pd.get_dummies(
   complete_df,
    columns=low_card,
    drop_first=True,
    sparse=True)

one_hot.head()

5. Feature Scaling

Feature Scaling("StandardScaler")

In [ ]:
from sklearn.preprocessing import StandardScaler
standard_df=complete_df.copy()
scaler=StandardScaler()
standard_df[num_cols]=scaler.fit_transform(complete_df[num_cols])
standard_df.head()


Feature Scaling("MinMaxScaler")

In [ ]:
from sklearn.preprocessing import MinMaxScaler
min_max_df=complete_df.copy()
MMS=MinMaxScaler()
min_max_df[num_cols]=MMS.fit_transform(complete_df[num_cols])
min_max_df.head()

6- Feature Enginnering

In [ ]:
df = pd.DataFrame(complete_df , columns = complete_df.columns)
df.isnull().sum()

-User Level Features

In [ ]:
user_total_orders = df.groupby('user_id')['order_number'].max().reset_index()
user_total_orders.columns = ['user_id', 'user_total_orders']#total_order
user_total_orders=user_total_orders.sort_values(by='user_id').reset_index(drop=True)
user_total_orders

In [ ]:
basket_data = df.groupby('user_id').agg({ 'product_id':'count', 'order_number': 'max'}).reset_index()
basket_data.columns = ['user_id', 'total_products_ordered', 'total_orders']
basket_data['avg_basket_size'] = basket_data['total_products_ordered'] / basket_data['total_orders']#avg basket sizehحجم سلة المتوسط يلي كل مره بشتري فيها 
basket_data.head()

In [ ]:
user_features = df.groupby('user_id').agg({
    'order_number': 'max',       
     'product_id': 'count',       
    'reordered': 'mean',         
    'days_since_prior_order': ['mean', 'last'] 
}).reset_index()
user_features.columns = ['user_id',  'user_total_orders',  'user_total_items', 'user_reorder_ratio','user_avg_days_between', 'user_days_since_last_order'  ]
display(user_features.head())


In [ ]:
user_features = df.groupby('user_id').agg({
    'order_number': 'max',       
     'product_id': 'count',       
    'reordered': 'mean',         
    'days_since_prior_order': ['mean', 'last'] 
}).reset_index()
user_features.columns = ['user_id',  'user_total_orders',  'total_products_ordered', 'user_reorder_ratio','user_avg_days_between', 'user_days_since_last_order'  ]
user_features['user_avg_basket_size'] = user_features[  'total_products_ordered'] / user_features['user_total_orders'] #size of basket
display(user_features.head())


- Product-level features

In [ ]:
product_features = df.groupby('product_id').agg({ 
        'reordered': 'mean', 
        'add_to_cart_order': 'mean',   
        'user_id': 'count'   #Popularity
}).reset_index()
product_features.columns = [ 'product_id' ,'product_reorder_rate', 'product_avg_cart_position', 'product_total_purchases']
display(product_features.head())

User×Product interaction features

In [ ]:

uxp_features = df.groupby(['user_id', 'product_id']).agg({
    'reordered': ['count', 'mean'], #prior purchase count& reorder probability عدد لمشتريات السابقة 
    'order_number': 'max'#last order
}).reset_index()  
uxp_features.columns = [
    'user_id', 
    'product_id', 
    'uxp_total_bought',       # كم مرة اشترى المنتج
    'uxp_reorder_ratio',      # نسبة إعادة الطلب
    'uxp_last_order_num'      # رقم آخر طلب اشترى فيه المنتج (عشان نحسب Recency بعدين)
]

display(uxp_features.head(20))

Temporal features

In [ ]:
df['order_hour']=df['order_hour_of_day']#Hour

df['order_day']=df['order_dow']#Day

start_year=2024
days_per_year=365
df['order_year'] = start_year + df['days_since_prior_order']//days_per_year #Year
df['order_month'] = ((df['days_since_prior_order'] % days_per_year) // 30) + 1 #Month

def get_season(month):
    if month in [3, 4, 5]:
        return 'Spring'
    elif month in [6, 7, 8]:
        return 'Summer'
    elif month in [9, 10, 11]:
        return 'Fall'
    else:
        return 'Winter'
df['order_season'] = df['order_month'].apply(get_season)#Season

df['is_weekend'] = df['order_dow'].apply(lambda x: 1 if x in [0, 1] else 0)#Holiday


In [ ]:
                                                                                              #جزء من temporal features ممكن انه نعملها 

def time_of_day(hour):
    if 6 <= hour < 11:                                                                        # بحيث انه اقسم الوقت صبح ومسا وهيك
        return 'Morning'
    elif 12 <= hour < 17:
        return 'Afternoon'
    elif 18 <= hour < 23:
        return 'Evening'
   
    else:
        return 'Night'
df['time_of_day'] = df['order_hour_of_day'].apply(time_of_day)
print(df['time_of_day'].head(10))

Aggregations over windows (e.g., last 3 orders, last 6 months).

In [ ]:

order_metrics = df.groupby(['user_id', 'order_number']).agg({
    'product_id': 'count',            # حجم السلة (كم حبة اشترى بالطلب)
    'days_since_prior_order': 'max'   # كم يوم غاب قبل هاد الطلب
}).reset_index()

order_metrics.rename(columns={
    'product_id': 'basket_size',
    'days_since_prior_order': 'days_gap'
}, inplace=True)

# لازم نرتب تصاعدي عشان لما نقول "آخر 3" يكونوا عنجد آخر 3
order_metrics = order_metrics.sort_values(['user_id', 'order_number'])

def get_recent_avg(x):
    return x.rolling(window=3, min_periods=1).mean()

order_metrics['avg_basket_last_3'] = order_metrics.groupby('user_id')['basket_size'].transform(get_recent_avg)#متوسط عدد المنتجات ل اخر 3 طلبات


order_metrics['avg_days_gap_last_3'] = order_metrics.groupby('user_id')['days_gap'].transform(get_recent_avg) #المتوسط الزمني بين الطلبات الثلاث الاخيرة 

user_rolling_features = order_metrics.groupby('user_id').last().reset_index() #احدث حالة لكل مستخدم 

user_rolling_features = user_rolling_features[['user_id', 'avg_basket_last_3', 'avg_days_gap_last_3']]


display(user_rolling_features.head(10))   

In [ ]:
train_df = df.copy()
train_df = train_df.merge(user_features , on = 'user_id' , how = 'left')
train_df = train_df.drop(columns=[col for col in user_features.columns if col in train_df.columns and col != 'user_id']) #تجنب التكرار
train_df = train_df.merge(user_features , on = 'user_id', how = 'left')
train_df = train_df.merge(product_features , on = 'product_id', how = 'left')
train_df = train_df.merge(uxp_features , on = ['user_id' , 'product_id'] , how = 'left')
train_df = train_df.merge(user_rolling_features , on = 'user_id' , how = 'left')

At least one engineered non-linear feature (log transforms, polynomial, interaction
terms).


In [ ]:
df['log_days_gap'] = np.log1p(df['days_since_prior_order'])
df[['days_since_prior_order', 'log_days_gap']].head(10)

Dimensionality & collinearity

In [ ]:
cols_for_corr = [
    'reordered', 
    'add_to_cart_order', 
    'order_number', 
    'order_dow', 
    'order_hour_of_day', 
    'days_since_prior_order'
]

sample_corr = complete_df[cols_for_corr].corr()
plt.figure(figsize=(10,8))
sns.heatmap(sample_corr, annot=True, cmap='coolwarm')
plt.title("Numerical Feature Correlation (Sampled Data)")
plt.show()

In [ ]:
id_cols = ['order_id' , 'user_id' , 'product_id' , 'aisle_id' , 'department_id']
low_cols = ["department_id" , "order_dow" , "time_of_day"] 
high_cols = ["user_id" , "product_id" , "aisle_id"] 

train_df[high_cols] = train_df[high_cols].astype(str)
#عشان التارقيت ما بشتغل غير مع كاتيقوري
target_col = "reordered"
Frequency_col = "product_name"

#عدد الاعمده كبير جدا فقلت بعمل لوب + استثناءات عشان اريح راسي
num_cols = (train_df.drop(columns=[target_col]).select_dtypes(include=["int32" , "float32"]).columns.tolist())
num_cols = [c for c in num_cols if c not in id_cols]

VIF

In [ ]:
reduce_memo(train_df)

In [ ]:
num_cols

In [ ]:
%pip install statsmodels
#بدي استخدم VIF , هاض عباره عن فنكشن رياضي ببين قديش في ارتباط وتكرار بين الاعمده نفسهم 
#الهدف منه اني اشوف شو في اعمده فيهم تشابه كبير وبقدمو نفس المعلومه تقريبا عشان احذف واحد منهم
#بدي استخدم VIF , هاض عباره عن فنكشن رياضي ببين قديش في ارتباط وتكرار بين الاعمده نفسهم 
#الهدف منه اني اشوف شو في اعمده فيهم تشابه كبير وبقدمو نفس المعلومه تقريبا عشان احذف واحد منهم
from statsmodels.stats.outliers_influence import variance_inflation_factor
SAMPLE_SIZE = 50000 
V = train_df[num_cols].sample(n=SAMPLE_SIZE , random_state = 42)

vif = pd.DataFrame()
vif["feature"] = V[num_cols].columns
vif["VIF"] = [variance_inflation_factor(V.values , i) for i in range(V.shape[1])]

#تحت 5 ممتاز
#بين ال 6 وال 10 مقبول
#اكثر من هيك بدك تشوف شو و وين في ترابط غير مهم وتبلش تحذف
#ال inf حذف مباشره

vif.sort_values("VIF" , ascending = True)

In [ ]:
plt.figure(figsize = (8 , 5))
plt.scatter(vif["VIF"] , vif["feature"])
plt.axvline(10 , color = 'red' , linestyle = '--')
plt.xlabel("VIF")
plt.title("VIF Scatter Plot")
plt.show()


In [ ]:
#حذفت القيم اللانهائيه , والقيم اللي فيها النسبه عاليه , لانهم بدلو على تكرار وتشابه المعلومات , يعني لو خليتهم كلهم زي كأني مكرر نفس العامود ما فرقت 
drop_cols = [
    
    "uxp_days_last_order_" ,
    #u_total_orders_log كنت بدي اخليه بما انه تعبنا عليه بمرحلة الهندسه بس النسبه فيه كانت كثير عاليه :(
    "user_total_orders" ,

    "product_avg_cart_position" ,
    "product_reorder_rate" ,
    "user_reorder_ratio" 
    ]

train_df = train_df.drop(columns = drop_cols , errors="ignore")

#صار عندي ايرور بالبريبروسيسر لانه الداتا صار فيها عدم تطابق بعد الدروب ف بدي ارجع انسخ الاعمده كمان مره
num_cols = (train_df.drop(columns=[target_col]).select_dtypes(include=["int32" , "float32"]).columns.tolist())
num_cols = [c for c in num_cols if c not in id_cols]

Embalanced Data Handling

In [ ]:
orders_train = pd.read_csv(r"C:\Users\Acer Nitro\Downloads\order_products__train.csv\order_products__train.csv")
orders_train = orders_train[['order_id', 'product_id', 'reordered']]

train_df['reordered_y']=orders_train['reordered']



In [ ]:
train_df['reordered_y'].value_counts()
train_df['reordered_y'].isnull().sum()

In [ ]:
train_df['reordered_y'].isnull().sum()
train_df['reordered_y'] = train_df['reordered_y'].fillna(0).astype(int)

In [ ]:
import matplotlib.pyplot as plt

fig, axes = plt.subplots(1, 2, figsize=(10, 4))

train_df['reordered'].value_counts().plot(
    kind='bar', ax=axes[0], title='reordered'
)

train_df['reordered_y'].value_counts().plot(
    kind='bar', ax=axes[1], title='reordered_y'
)

plt.tight_layout()
plt.show()


هيك بنكون بيننا انو اصلا في عنا imbalanced بالداتا

In [ ]:
reduce_memo(train_df)

In [ ]:
train_df=train_df.drop('reordered_y',axis=1)

TO SMOTE

In [ ]:
id_cols = ['order_id', 'user_id', 'product_id', 'aisle_id', 'department_id']
low_cols_ = ["department", "order_dow" , "time_of_day", "aisle", "order_season"] 
high_cols = ["user_id", "product_id", "aisle_id"] 

df[high_cols] = df[high_cols].astype(str)
# عشان التارقيت ما بوخذ غير category

target_col = "reordered_y"
Frequency_col = "product_name"
num_cols_ = [
    
    'user_avg_basket',
    'user_reorder_ratio',
    'user_last_order_recency',
    'product_popularity',
    'user_prod_purchase_count'
]

In [ ]:
# Ensure category_encoders is installed in the current kernel (try both PyPI names to be robust)
%pip install --quiet category_encoders


# import after installation
import category_encoders as ce

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler

# Frequency_col is a single column name string — pass as a list to ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ("onehot",
         OneHotEncoder(handle_unknown="ignore", sparse_output=True, drop="first"),
         low_cols),

        ("target",
         ce.TargetEncoder(min_samples_leaf=20, smoothing=50),
         high_cols),

        ("freq",
         ce.CountEncoder(normalize=True),
         [Frequency_col]),

        ("scale",
         StandardScaler(),
         num_cols)
    ],
    remainder="passthrough"
)


In [ ]:
!pip install imbalanced-learn
!pip install xgboost
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as ImbPipeline #  استدعينا بايبلاين خاص
from xgboost import XGBClassifier

# 1. تعريف الـ SMOTE
# sampling_strategy=0.5 معناها: خلي عدد الـ 1 يصير نص عدد الـ 0 (نسبة 1:2)
# هيك بنحسن التوازن بدون ما نغرق الداتا ببيانات صناعية
smote = SMOTE(random_state=42, sampling_strategy=0.5)

# 2. تعريف الموديل
# ملاحظة مهمة: شلنا scale_pos_weight أو خليناه 1
# ليش؟ لأن SMOTE أصلاً حل مشكلة الـ Imbalance، فما في داعي نعاقب الموديل مرتين!
clf = XGBClassifier(
    n_estimators=1000,
    learning_rate=0.05,
    max_depth=6,
    scale_pos_weight=1,  # رجعناه للطبيعي
    subsample=0.8,
    colsample_bytree=0.8,
    n_jobs=-1,
    random_state=42,
    eval_metric='auc'
)

# 3. بناء البايبلاين الجديد (ImbPipeline)
# الترتيب: معالجة -> SMOTE -> موديل
model_pipeline = ImbPipeline(steps=[
    ('preprocessor', preprocessor), # معالجتك القديمة
    ('smote', smote),               # خطوة التوليد (بتصير بس عالـ Train)
    ('model', clf)                  # التدريب
])



In [ ]:

from sklearn.model_selection import KFold
target_users_count = 10000 

unique_users = train_df['user_id'].unique()

if target_users_count > len(unique_users):
    target_users_count = len(unique_users)

np.random.seed(42)
my_users = np.random.choice(unique_users , size=target_users_count , replace=False)

final_sample = train_df[train_df['user_id'].isin(my_users)].copy()

final_sample = final_sample.sort_values(by=["user_id", "order_number"])
final_sample["last__orders"] = final_sample["order_number"] == final_sample.groupby("user_id")["order_number"].transform("max")

train_df_ = final_sample[final_sample['last__orders'] == False]
test_df_ = final_sample[final_sample['last__orders'] == True]

not_for_X_columns = ['reordered' , 'eval_set' , 'last__orders' ,  'order_id' , 'add_to_cart_order' , 'order_number',  'aisle' , 'department','time_of_day','uxp_last_order_num'] 

xc_train = train_df_.drop(columns=not_for_X_columns , errors='ignore')
yc_train = train_df_['reordered']

xc_test = test_df_.drop(columns = not_for_X_columns , errors = 'ignore')
yc_test = test_df_['reordered']

real_num_cols = [c for c in num_cols if c not in not_for_X_columns]
real_low_cols = [c for c in low_cols if c not in not_for_X_columns]
real_high_cols = [c for c in high_cols if c not in not_for_X_columns]

# مدامني قسمت الداتا حسب الزمن فهيك عالاغلب مش رح احتاجه
KF = KFold(n_splits = 5 , shuffle = True , random_state = 42)

preprocessor = ColumnTransformer(
    transformers=[
        ("encoding" , OneHotEncoder(handle_unknown = "ignore" , sparse_output = True , drop = "first") , real_low_cols) ,

        # Target Encoding يُستخدم مع الأعمدة الفئوية ذات عدد القيم الكبير.
        # يتم تحويل كل فئة إلى متوسط قيمة المتغير الهدف المرتبط بها.
        # لتجنب تسريب الهدف (Target Leakage)، يتم تطبيق الترميز داخل
        # الـ Cross-Validation بحيث يُحسب الترميز من بيانات التدريب فقط.
        # معاملات min_samples_leaf و smoothing تقلل تأثير الفئات النادرة
        # عبر تقريبها من المتوسط العام، مما يحد من الـ overfitting.
        ("target_encoding" , ce.TargetEncoder(min_samples_leaf = 20 , smoothing = 50) , real_high_cols) ,
        ("Frequency" , ce.CountEncoder(normalize = True) , Frequency_col) ,
        ("scaling" , StandardScaler() , real_num_cols)
                 ]
)

# normlize =True -> عشان يخليلي كل القيم بين 0 و 1

train_cls = preprocessor.fit_transform(xc_train , yc_train)
test_cls = preprocessor.transform(xc_test)

Tuning

In [ ]:
from sklearn.model_selection import TimeSeriesSplit
np.random.seed(42)
tuning_x = xc_train.copy()
tuning_y = yc_train.copy()

from sklearn.model_selection import GroupKFold

gkf = GroupKFold(n_splits=3)


KNN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV
KNNsmall_users = np.random.choice(tuning_x['user_id'].unique() , size = 2000 , replace = False)

knn_x_small = tuning_x[tuning_x['user_id'].isin(KNNsmall_users)].copy()
knn_y_small = tuning_y[tuning_x['user_id'].isin(KNNsmall_users)]



KNN_pipline = make_pipeline(
    preprocessor ,
    KNeighborsClassifier(n_jobs = -1)
)

KNNparam_grid = {
    "kneighborsclassifier__n_neighbors": [7 , 10 , 30,60,80,150] ,
    "kneighborsclassifier__weights" : ['distance' , 'uniform']
}

KNN_grid_search = GridSearchCV(
    estimator = KNN_pipline ,
    param_grid = KNNparam_grid ,
    cv = gkf ,
    scoring = "f1" ,
    n_jobs = -1
)

KNN_grid_search.fit(
    knn_x_small,
    knn_y_small,
    groups=knn_x_small['user_id']
)

print("\nBest Score:", KNN_grid_search.best_score_)
print("Best Parameters:", KNN_grid_search.best_params_)
'''with k =60 and weights = uniform got best score of 0.87'''
'''with k =30 and weights = uniform got best score of 0.8974819271262918'''
'''with k =7 and weights = distance got best score of 0.9720661157'''


In [ ]:
best_w = KNN_grid_search.best_params_['kneighborsclassifier__weights']
best_k = KNN_grid_search.best_params_['kneighborsclassifier__n_neighbors']

finalKNN_pip = make_pipeline(
        preprocessor , 
        KNeighborsClassifier(n_neighbors = best_k , weights = 'uniform' , n_jobs = -1, metric='manhattan', )
)

finalKNN_pip.fit(xc_train, yc_train)


y_pred_KNN = finalKNN_pip.predict(xc_test)

print(classification_report(yc_test , y_pred_KNN))
print('KNN Accuracy',accuracy_score(yc_test , y_pred_KNN))

'''With Best Parameters k=30 , metric=distance : 0.9433722102'''
''' With Tuning and change metric and wights : 0.8974819271262918 '''
'''When i increase k over 30 the accuracy increases'''
'''k=80 got accuracy of 0.9123456790123457 > k=30 '''



Logistic Regression

In [ ]:
from sklearn.model_selection import RandomizedSearchCV


tuning_users = np.random.choice(tuning_x["user_id"].unique(), size=2500, replace=False) 

# تجهيز عينة التونينق
LoR_x_small = tuning_x[tuning_x['user_id'].isin(tuning_users)].copy().sort_values(by=['user_id'])
LoR_y_small = tuning_y.loc[LoR_x_small.index]

LoR_pipline = make_pipeline(
    preprocessor , 
    LogisticRegression(solver = 'liblinear' , class_weight = 'balanced' , random_state = 42 , n_jobs = -1)
)

LoRparam_grid = {
    "logisticregression__C": [0.001 , 0.01 , 0.1 , 1 , 10] ,
    "logisticregression__penalty": ['l1' , 'l2']
}

LoR_grid_search = GridSearchCV(
    estimator = LoR_pipline ,
    param_grid = LoRparam_grid ,
    cv = gkf ,
    scoring = "f1" ,
    n_jobs = -1
)

LoR_grid_search.fit(LoR_x_small , LoR_y_small, groups=LoR_x_small['user_id'])

print("\nBest Score:", LoR_grid_search.best_score_)
print("Best Parameters:", LoR_grid_search.best_params_)







In [ ]:
best_C = LoR_grid_search.best_params_["logisticregression__C"]
best_P = LoR_grid_search.best_params_["logisticregression__penalty"]

final_LoR_pipeline = make_pipeline(
    preprocessor , 
    LogisticRegression(solver = 'liblinear' , C = best_C , penalty = best_P , class_weight = 'balanced' , random_state = 42 , n_jobs = -1)
)

final_LoR_pipeline.fit(xc_train, yc_train)

y_pred_LoR = final_LoR_pipeline.predict(xc_test)

print(classification_report(yc_test, y_pred_LoR))
print(f"Logistic Accuracy: {accuracy_score(yc_test , y_pred_LoR):.4f}")

Random Forest

In [ ]:

from sklearn.ensemble import RandomForestClassifier


tuning_users = np.random.choice(tuning_x["user_id"].unique(), size=800, replace=False)
RF_x_small = tuning_x[tuning_x['user_id'].isin(tuning_users)].copy().sort_values(by=['user_id'])
RF_y_small = tuning_y.loc[RF_x_small.index]

RF_pipline = make_pipeline(
    preprocessor ,
    RandomForestClassifier(class_weight = "balanced" , random_state = 42 , n_jobs = -1)
)

RFparam_grid = {
    "randomforestclassifier__n_estimators": [100 , 150 , 200] , 
    "randomforestclassifier__max_depth": [None , 10 , 20] , 
    "randomforestclassifier__min_samples_split": [2 , 5 , 10] ,
    "randomforestclassifier__min_samples_leaf": [1 , 2 , 4] ,
    "randomforestclassifier__class_weight": ['balanced' , 'balanced_subsample']
}

RF_grid_search = GridSearchCV(
    estimator = RF_pipline ,
    param_grid = RFparam_grid , 
    cv =gkf,
    scoring = "f1" ,
    n_jobs = -1
)

RF_grid_search.fit(RF_x_small , RF_y_small,groups=RF_x_small['user_id'])

print("\nBest Score:", RF_grid_search.best_score_)
print("Best Parameters:", RF_grid_search.best_params_)

In [ ]:
best_N = RF_grid_search.best_params_["randomforestclassifier__n_estimators"]
best_MD = RF_grid_search.best_params_["randomforestclassifier__max_depth"]
best_MSP = RF_grid_search.best_params_[ "randomforestclassifier__min_samples_split"]
best_MSL = RF_grid_search.best_params_["randomforestclassifier__min_samples_leaf"]
best_W = RF_grid_search.best_params_["randomforestclassifier__class_weight"]

final_RF_pipeline = make_pipeline(
    preprocessor,
    RandomForestClassifier(n_estimators = best_N , max_depth = best_MD , 
                    min_samples_leaf = best_MSL , min_samples_split = best_MSP , 
                 class_weight = best_W , n_jobs = -1 , random_state = 42 
                        )       
)



final_RF_pipeline.fit(xc_train , yc_train)

y_pred_RF = final_RF_pipeline.predict(xc_test)

print(classification_report(yc_test, y_pred_RF))
print(f"Random Forest Accuracy: {accuracy_score(yc_test, y_pred_RF):.4f}")

Support Vector Machine 

In [ ]:
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import RandomizedSearchCV, GroupKFold
import numpy as np
svm_users = np.random.choice(
    xc_train['user_id'].unique(),
    size=1000,
    replace=False
)

svm_x_small = xc_train[xc_train['user_id'].isin(svm_users)].copy()
svm_y_small = yc_train[xc_train['user_id'].isin(svm_users)]

svm_pipeline = make_pipeline(
    preprocessor,
    SVC(probability=True, random_state=42)
)

from sklearn.model_selection import RandomizedSearchCV, GroupKFold
import numpy as np

param_dist = {
    "svc__C": np.logspace(-2, 2, 6),
    "svc__kernel": ["rbf", "linear"],
    "svc__gamma": ["scale", "auto"]
}

gkf = GroupKFold(n_splits=3)

svm_random = RandomizedSearchCV(
    estimator=svm_pipeline,
    param_distributions=param_dist,
    n_iter=6,
    scoring="roc_auc",
    cv=gkf,
    n_jobs=1,
    random_state=42,
    verbose=1
)

svm_random.fit(
    svm_x_small,
    svm_y_small,
    groups=svm_x_small['user_id']
)

print('Best Score :',svm_random.best_score_)
print('Best Parameters :',svm_random.best_params_)

In [ ]:
best_svm = svm_random.best_estimator_

from sklearn.metrics import accuracy_score, roc_auc_score, classification_report

y_pred_svm = best_svm.predict(xc_test)
y_prob_svm = best_svm.predict_proba(xc_test)[:, 1]

accuracy_score(yc_test, y_pred_svm)
roc_auc_score(yc_test, y_prob_svm)
print(classification_report(yc_test, y_pred_svm))


Linear SVM


In [ ]:
dt_features = [
    'order_dow',
    'order_hour_of_day',
    'days_since_prior_order',
    'is_weekend',
    'order_month',
    'user_total_items',
    'user_avg_days_between',
    'user_days_since_last_order',
    'product_total_purchases',
    'uxp_total_bought',
    'uxp_reorder_ratio',
    'avg_basket_last_3',
    'avg_days_gap_last_3'
]


In [ ]:


from sklearn.svm import LinearSVC
from sklearn.calibration import CalibratedClassifierCV
from sklearn.pipeline import make_pipeline
svm_users = np.random.choice(
    xc_train['user_id'].unique(),
    size=5000,
    replace=False
)

svm_x_small = xc_train[xc_train['user_id'].isin(svm_users)][dt_features].copy()
svm_y_small = yc_train[xc_train['user_id'].isin(svm_users)]

svm_sample_pipeline = make_pipeline(
    CalibratedClassifierCV(
        LinearSVC(
            C=10,
            max_iter=5000,
            class_weight="balanced",
            random_state=42
        ),
        method="sigmoid"
    )
)
svm_sample_pipeline.fit(svm_x_small, svm_y_small)
print("SVM with Linear Kernel Accuracy:", accuracy_score(svm_y_small, svm_sample_pipeline.predict(svm_x_small)))
print(classification_report(svm_y_small, svm_sample_pipeline.predict(svm_x_small)))

'''SVM Score with Linear Kernel Accuracy: 0.809161776248863 -> with c= 1.0'''
'''SVM Score with Linear Kernel Accuracy: 0.7996820567076329 -> with c= 10'''
'''SVM Score with Linear Kernel Accuracy: 0.8020444783838858 -> with c= 0.1'''
'''SVM Score with Linear Kernel Accuracy: 0.7984317144654071 with c=100'''

'''The best Score C is 1.0 as it gives the highest accuracy of 0.8091'''


In [ ]:
dt_features = [
    'order_dow',
    'order_hour_of_day',
    'days_since_prior_order',
    'is_weekend',
    'order_month',
    'user_total_items',
    'user_avg_days_between',
    'user_days_since_last_order',
    'product_total_purchases',
    'uxp_total_bought',
    'uxp_reorder_ratio',
    'avg_basket_last_3',
    'avg_days_gap_last_3'
]


x_test_svm = xc_test[dt_features]

y_pred_svm = svm_sample_pipeline.predict(x_test_svm)

print("SVM Accuracy:", accuracy_score(yc_test, y_pred_svm))
print(classification_report(yc_test, y_pred_svm))
print (f"SVM Accuracy: {accuracy_score(yc_test , y_pred_svm):.4f}")
'''SVM Accuracy 0.7327 With C=1.0'''
'''SVM Accuracy 0.7273 With C=10'''


Decision Tree

In [ ]:
dt_features = [
    'order_dow',
    'order_hour_of_day',
    'days_since_prior_order',
    'is_weekend',
    'order_month',
    'user_total_items',
    'user_avg_days_between',
    'user_days_since_last_order',
    'product_total_purchases',
    'uxp_total_bought',
    'uxp_reorder_ratio',
    'avg_basket_last_3',
    'avg_days_gap_last_3'
]

X_train_dt = xc_train[dt_features]
X_test_dt = xc_test[dt_features]





In [ ]:
dt_users = np.random.choice(
    xc_train['user_id'].unique(),
    size=5000,
)

dt_x_sample = xc_train[xc_train['user_id'].isin(dt_users)].copy()
dt_y_sample = yc_train[xc_train['user_id'].isin(dt_users)]


In [ ]:
best_MD = 50
best_MSL = 20
best_MSS = 20
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import make_pipeline
dt_model = DecisionTreeClassifier(
    max_depth=5,
    min_samples_leaf=5,
    min_samples_split=5,
    class_weight='balanced',
    random_state=42
)
final_DT_pipeline = make_pipeline(
    preprocessor,
    DecisionTreeClassifier(
        max_depth=best_MD,
        min_samples_leaf=best_MSL,
        min_samples_split=best_MSS,
        class_weight="balanced",
        random_state=42
    )
)
dt_model.fit(X_train_dt, yc_train)


In [ ]:
from sklearn.metrics import accuracy_score, classification_report

y_pred_DT = dt_model.predict(X_test_dt)

print(classification_report(yc_test, y_pred_DT))
print(f"Decision Tree Accuracy: {accuracy_score(yc_test, y_pred_DT):.4f}")


'''Decision Tree Accuracy: 0.8300 with max_depth=30, min_samples_leaf=10, min_samples_split=20'''
'''Decision Tree Accuracy: 0.8330 with max_depth=20, min_samples_leaf=10, min_samples_split=10'''
'''Decision Tree Accuracy: 0.8331 with max_depth=50, min_samples_leaf=20, min_samples_split=10'''
'''Decision Tree Accuracy: 0.8172 with max_depth=5, min_samples_leaf=5, min_samples_split=5  '''


XGBOOST

In [ ]:
!pip install lightgbm 
!pip install xgboost
gb_features = [
    'order_dow',
    'order_hour_of_day',
    'days_since_prior_order',
    'is_weekend',
    'order_month',
    'user_total_items',
    'user_avg_days_between',
    'user_days_since_last_order',
    'product_total_purchases',
    'uxp_total_bought',
    'uxp_reorder_ratio',
    'avg_basket_last_3',
    'avg_days_gap_last_3'
]

gb_users = np.random.choice(
    xc_train['user_id'].unique(),
    size=5000,
    replace=False
)

X_gb_sample = xc_train[xc_train['user_id'].isin(gb_users)][gb_features]
y_gb_sample = yc_train[xc_train['user_id'].isin(gb_users)]

X_gb_test = xc_test[gb_features]


In [ ]:
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report

xgb_model = XGBClassifier(
    n_estimators=300,
    max_depth=100,
    learning_rate=0.1,
    subsample=0.8,
    colsample_bytree=0.8,
    scale_pos_weight=(len(y_gb_sample) - y_gb_sample.sum()) / y_gb_sample.sum(),
    objective='binary:logistic',
    eval_metric='logloss',
    random_state=42,
    n_jobs=-1
)

xgb_model.fit(X_gb_sample, y_gb_sample)

y_pred_xgb = xgb_model.predict(X_gb_test)

print(classification_report(yc_test, y_pred_xgb))
print(f"XGBoost Accuracy: {accuracy_score(yc_test, y_pred_xgb):.4f}")

'''XGBoost Accuracy: 0.8253 with n_estimators=300, max_depth=6 , learning_rate=0.05'''

'''XGBoost Accuracy: 0.8645 with n_estimators=300, max_depth=20 , learning_rate=0.1'''

'''XGBoost Accuracy: 0.8637 with n_estimators=300, max_depth=50 , learning_rate=0.1'''


LIGHT GBM

In [ ]:
from lightgbm import LGBMClassifier

lgbm_model = LGBMClassifier(
    n_estimators=300,
    max_depth=20,
    learning_rate=0.1,
    subsample=0.8,
    colsample_bytree=0.8,
    class_weight='balanced',
    random_state=42,
    n_jobs=-1
)

lgbm_model.fit(X_gb_sample, y_gb_sample)

y_pred_lgbm = lgbm_model.predict(X_gb_test)

print(classification_report(yc_test, y_pred_lgbm))
print(f"LightGBM Accuracy: {accuracy_score(yc_test, y_pred_lgbm):.4f}")

''''LightGBM Accuracy: 0.8233 with n_estimators=300, max_depth=6 , learning_rate=0.05'''

'''LightGBM Accuracy: 0.8255 with n_estimators=300, max_depth=20 , learning_rate=0.1'''


BOUNS AUTO ML 

CLASSIFICATION DONE ✅ TASK 1 ✅